In [89]:
import pandas as pd
import numpy as np
import yfinance as yf
import math


def load_data(path):
    results_models_comparison = pd.read_csv(f"{path}/data/cleaned/predictions.csv")
    df_inventory = pd.read_excel(f"{path}/data/raw/catusita/inventory.xlsx")
    tipo_de_cambio_df = pd.read_excel(f"{path}/data/raw/catusita/saldo de todo 04.11.2024.2.xls", skiprows=2)
    df_products = pd.read_csv(f"{path}/data/process/catusita_consolidated.csv")
    try:
        back_order = pd.read_excel(f"{path}/data/raw/catusita/backorder12_12.xlsx")
    except FileNotFoundError:
        back_order = "skippable"
    return results_models_comparison, df_inventory, tipo_de_cambio_df, df_products, back_order

path = 'C:/Users/Christopher/OneDrive/Documentos/GitHub/catusita_revamp'  # Replace with your actual path
results_models_comparison, df_inventory, tipo_de_cambio_df, df_products, back_order = load_data(path)

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
from typing import Tuple, Optional

class DataProcessor:
    def __init__(self, path: str):
        self.path = path
        self.results_models_comparison = None
        self.df_inventory = None
        self.tipo_de_cambio_df = None
        self.df_products = None
        self.back_order = None
        self.closing_prices = None
        self.long_format = None
        self.merged_df_tc_final = None
        self.df_merged = None
        self.result_precio = None
        self.margin_result = None
        self.df1_final = None
        self.dffinal2 = None

    def load_data(self) -> None:
        self.results_models_comparison = pd.read_csv(f"{self.path}/data/cleaned/predictions.csv")
        self.df_inventory = pd.read_excel(f"{self.path}/data/raw/catusita/inventory.xlsx")
        self.tipo_de_cambio_df = pd.read_excel(f"{self.path}/data/raw/catusita/saldo de todo 04.11.2024.2.xls", skiprows=2)
        self.df_products = pd.read_csv(f"{self.path}/data/process/catusita_consolidated.csv")
        try:
            self.back_order = pd.read_excel(f"{self.path}/data/raw/catusita/backorder12_12.xlsx")
        except FileNotFoundError:
            self.back_order = pd.DataFrame()

    def preprocess_exchange_rates(self) -> None:
        self.tipo_de_cambio_df = self.tipo_de_cambio_df[['Código','Mnd','Fob','Ult. Fecha','Ult. Compra']]
        self.tipo_de_cambio_df.columns = ['codigo', 'moneda', 'monto', 'ultima_fecha', 'ultima_compra']
        self.tipo_de_cambio_df = self.tipo_de_cambio_df.copy()
        self.tipo_de_cambio_df['codigo'] = self.tipo_de_cambio_df['codigo'].astype(str)
        self.tipo_de_cambio_df = self.tipo_de_cambio_df.dropna(subset=['ultima_fecha'])
        self.tipo_de_cambio_df['codigo'] = self.tipo_de_cambio_df['codigo'].str.lower()
        self.tipo_de_cambio_df = self.tipo_de_cambio_df[self.tipo_de_cambio_df['ultima_fecha'].notna()]
        self.tipo_de_cambio_df['ultima_fecha'] = pd.to_datetime(self.tipo_de_cambio_df['ultima_fecha'], errors='coerce')

    def get_currency_data(self) -> None:
        start = self.tipo_de_cambio_df['ultima_fecha'].min().date()
        end = self.tipo_de_cambio_df['ultima_fecha'].max().date()
        currency_pairs = ['PENUSD=X', 'EURUSD=X', 'JPYUSD=X', 'GBPUSD=X']
        data = yf.download(currency_pairs, start=start, end=end)
        self.closing_prices = data['Close']
        self.closing_prices.columns = [col.split('.')[0] for col in self.closing_prices.columns]

    def process_currency_data(self) -> None:
        self.long_format = self.closing_prices.reset_index().melt(id_vars='Date', var_name='Currency Pair', value_name='Closing Price')
        self.long_format['Currency Pair'] = self.long_format['Currency Pair'].str.replace('=X', '', regex=False)
        self.long_format = self.long_format.dropna(subset=['Closing Price'])
        
        full_date_range = pd.date_range(start=self.long_format['Date'].min(), end=self.long_format['Date'].max(), freq='D')
        currency_pairs = self.long_format['Currency Pair'].unique()
        complete_index = pd.MultiIndex.from_product([full_date_range, currency_pairs], names=['Date', 'Currency Pair'])
        df_full = pd.DataFrame(index=complete_index).reset_index()
        
        self.long_format = df_full.merge(self.long_format, on=['Date', 'Currency Pair'], how='left')
        self.long_format['Closing Price'] = self.long_format.groupby('Currency Pair')['Closing Price'].fillna(method='ffill')
        self.long_format = self.long_format.rename(columns={'Closing Price': 'tc'})

    def merge_exchange_rates(self) -> None:
        merged_df_tc = pd.merge(self.tipo_de_cambio_df, self.long_format, left_on='ultima_fecha', right_on='Date', how='left')
        merged_df_tc['monto'] = pd.to_numeric(merged_df_tc['monto'], errors='coerce')
        merged_df_tc['tc'] = pd.to_numeric(merged_df_tc['tc'], errors='coerce')
        
        def convert_to_usd(row):
            if pd.isna(row['Currency Pair']) or row['moneda'] == 'USD':
                return row['monto']
            currency_pair_map = {'SOL': 'PENUSD', 'EUR': 'EURUSD', 'JPY': 'JPYUSD', 'GBP': 'GBPUSD'}
            if row['moneda'] in currency_pair_map and row['Currency Pair'] == currency_pair_map[row['moneda']]:
                return row['monto'] / row['tc'] if row['moneda'] == 'SOL' else row['monto'] * row['tc']
            return 0

        merged_df_tc['monto_usd'] = merged_df_tc.apply(convert_to_usd, axis=1)
        merged_df_tc = merged_df_tc[merged_df_tc['monto_usd'] != 0]
        self.merged_df_tc_final = merged_df_tc[['codigo', 'ultima_fecha', 'monto_usd', 'ultima_compra']]
        self.merged_df_tc_final = self.merged_df_tc_final[self.merged_df_tc_final['monto_usd'].notna()]

    def process_inventory(self) -> None:
        self.df_inventory = self.df_inventory.copy()
        self.df_inventory.columns = ['cia', 'date', 'codigo', 'descripcion', 'um', 'stock']
        
        self.df_inventory = self.df_inventory[
            (self.df_inventory['date'] != 'Periodo') & 
            (self.df_inventory['date'].notna())
        ]

        self.df_inventory['date'] = pd.to_datetime(self.df_inventory['date'], format='%d/%m/%Y')
        max_date = self.results_models_comparison['date'].max()
        self.df_inventory = self.df_inventory[
            (self.df_inventory['date'] == max_date) & 
            (self.df_inventory['codigo'].notna())
        ]
        self.df_inventory.loc[:, 'codigo'] = self.df_inventory['codigo'].str.lower()

    def merge_dataframes(self) -> None:
        self.df_merged = self.results_models_comparison.copy()
        self.df_merged = self.df_merged.rename(columns={'sku':'articulo'})
        self.df_merged = self.df_merged.merge(
            self.df_products[['articulo', 'fuente_suministro','lt']].drop_duplicates(), 
            how='left', 
            on='articulo'
        )
        self.df_merged['date'] = pd.to_datetime(self.df_merged['date'])
        self.df_inventory['date'] = pd.to_datetime(self.df_inventory['date'])
        self.df_merged = self.df_merged.merge(
            self.df_inventory[['codigo', 'stock', 'date']].drop_duplicates(), 
            how='left', 
            left_on=['articulo', 'date'], 
            right_on=['codigo', 'date']
        )
        self.df_merged['stock'] = self.df_merged['stock'].fillna(0)
        self.df_merged = self.df_merged.drop(columns='codigo')

    def calculate_risk(self) -> None:
        self.df_merged['index_riesgo'] = self.df_merged['stock'] / (self.df_merged['caa'] / self.df_merged['lt_x'])
        self.df_merged['riesgo'] = pd.cut(
            self.df_merged['index_riesgo'], 
            bins=[-float('inf'), 1, 1.2, 1.5, float('inf')],
            labels=['Rojo', 'Naranja', 'Amarillo', 'Verde'], 
            right=False
        )

    def process_prices(self) -> None:
        df_precio = self.df_products[['articulo', 'cantidad', 'venta_pen', 'fecha']].copy()
        df_precio['fecha'] = pd.to_datetime(df_precio['fecha'], errors='coerce')
        df_precio = df_precio[df_precio['fecha'].dt.year == 2024]
        df_precio['precio'] = df_precio['venta_pen'] / df_precio['cantidad']
        self.result_precio = df_precio.groupby('articulo').agg(precio=('precio', 'mean')).reset_index()

    def calculate_margin(self) -> None:
        df_margen = self.df_products[['articulo', 'costo', 'venta_pen', 'fecha']].copy()
        df_margen['fecha'] = pd.to_datetime(df_margen['fecha'], errors='coerce')
        df_margen = df_margen[df_margen['fecha'].dt.year == 2024]
        df_margen['margen'] = df_margen['venta_pen'] / df_margen['costo'] - 1
        self.margin_result = df_margen.groupby('articulo').agg(
            total_venta_pen=('venta_pen', 'sum'),
            mean_margen=('margen', 'mean')
        ).reset_index().sort_values(by='total_venta_pen', ascending=False)

    def create_df1_final(self) -> None:
        df1 = self.df_merged[['fuente_suministro', 'date', 'articulo','real', 'catusita', 'caa','lt_x']].copy()
        df1 = df1.rename(columns={'catusita': 'venta_sin_recomendacion', 'caa': 'venta_con_recomendacion'})
        self.df1_final = df1.merge(self.result_precio, how='left', on='articulo')
        self.df1_final = self.df1_final[['fuente_suministro', 'date', 'articulo', 'venta_sin_recomendacion', 'venta_con_recomendacion','real', 'precio','lt_x']]
        
        self.df1_final['ingreso_sin_recomendacion'] = np.where(
            self.df1_final['venta_sin_recomendacion'] < self.df1_final['real'],
            self.df1_final['venta_sin_recomendacion'] * self.df1_final['precio'],
            self.df1_final['real'] * self.df1_final['precio']
        )
        
        self.df1_final['venta_con_recomendacion'] = np.where(
            self.df1_final['venta_con_recomendacion'] < self.df1_final['real'],
            self.df1_final['venta_con_recomendacion'] * self.df1_final['precio'],
            self.df1_final['real'] * self.df1_final['precio']
        )
        
        self.df1_final['ingreso_sin_recomendacion_ajustado'] = self.df1_final['ingreso_sin_recomendacion'] / (self.df1_final['lt_x'] * 0.83)
        self.df1_final['ingreso_con_recomendación_ajustado'] = self.df1_final['venta_con_recomendacion'] / (self.df1_final['lt_x'] * 0.83)
        
        penusd_tc = self.long_format[self.long_format['Currency Pair'] == 'PENUSD'].groupby('Date')['tc'].last().reset_index()
        self.df1_final = self.df1_final.merge(penusd_tc, how='left', left_on='date', right_on='Date')
        self.df1_final['tc'] = 1/self.df1_final['tc']
        self.df1_final['ingreso_usd_sin_recomendacion'] = self.df1_final['ingreso_sin_recomendacion_ajustado'] / self.df1_final['tc']
        self.df1_final['ingreso_usd_con_recomendacion'] = self.df1_final['ingreso_con_recomendación_ajustado'] / self.df1_final['tc']
        self.df1_final = self.df1_final[['fuente_suministro', 'date', 'articulo', 'lt_x', 'ingreso_usd_sin_recomendacion', 'ingreso_usd_con_recomendacion', 'tc']]
        self.df1_final = self.df1_final.drop_duplicates()

    def create_final_dataframe(self) -> None:
        last_date = self.df_merged['date'].max()
        df_merged_last = self.df_merged[self.df_merged['date'] == last_date].copy()
        
        df_merged_last['demanda_mensual'] = df_merged_last['caa'] / df_merged_last['lt_x']
        self.dffinal2 = df_merged_last[['articulo', 'stock', 'caa', 'demanda_mensual', 'corr_sd', 'index_riesgo', 'riesgo', 'lt_x']]
        self.dffinal2 = self.dffinal2.copy()
        self.dffinal2['meses_proteccion'] = self.dffinal2['corr_sd'] / self.dffinal2['demanda_mensual']
        self.dffinal2 = self.dffinal2[['articulo', 'stock', 'caa', 'demanda_mensual', 'meses_proteccion', 'index_riesgo', 'riesgo', 'lt_x']]
        self.dffinal2 = self.dffinal2.merge(self.margin_result[['articulo', 'mean_margen']], how='left', on='articulo')
        self.dffinal2 = self.dffinal2.merge(self.merged_df_tc_final, how='left', left_on='articulo', right_on='codigo')

    def add_compra_real(self) -> None:
        df_predicciones = pd.read_csv(f"{self.path}/data/cleaned/predictions.csv")
        df_inventory2 = pd.read_excel(f"{self.path}/data/raw/catusita/inventory.xlsx")
        
        df_predicciones = df_predicciones.rename(columns={'sku': 'articulo'})
        
        df_inventory2 = df_inventory2[
            (df_inventory2['FECHA AL'] != 'Periodo') & 
            (df_inventory2['FECHA AL'].notna())
        ]
        
        df_inventory2['FECHA AL'] = pd.to_datetime(df_inventory2['FECHA AL'], format='%d/%m/%Y')
        max_date = df_inventory2['FECHA AL'].max()
        df_inventory2 = df_inventory2[df_inventory2['FECHA AL'] == max_date]
        df_inventory2['FECHA AL'] = df_inventory2['FECHA AL'].dt.strftime('%d/%m/%Y')
        df_inventory2['CODIGO'] = df_inventory2['CODIGO'].str.lower()
        
        df_predicciones['date'] = pd.to_datetime(df_predicciones['date'], format='%Y-%m-%d')
        df_predicciones['date'] = df_predicciones['date'].dt.strftime('%d/%m/%Y')
        
        merged_df = df_predicciones.merge(
            df_inventory2[['FECHA AL', 'CODIGO', 'STOCK']], 
            how='left', 
            left_on=['articulo'], 
            right_on=['CODIGO']
        )
        
        merged_df['STOCK'] = merged_df['STOCK'].fillna(0)
        
        if not self.back_order.empty:
            merged_df = merged_df.merge(self.back_order, how='left', on='articulo')
            merged_df['backorder'] = merged_df['backorder'].fillna(0)
        else:
            merged_df['backorder'] = 0
        
        merged_df['sobrante'] = np.maximum(merged_df['STOCK'] + merged_df['backorder'] - merged_df['caa_lt'], 0)
        merged_df['nueva_compra_sugerida'] = np.maximum(merged_df['caa'] - merged_df['sobrante'], 0)
        merged_df['nueva_compra_sugerida'] = np.ceil(merged_df['nueva_compra_sugerida']).astype(int)
        
        merge_columns = merged_df[['articulo', 'nueva_compra_sugerida', 'caa', 'backorder']].copy()
        
        self.dffinal2 = self.dffinal2.merge(merge_columns, how='left', on='articulo')
        self.dffinal2['compra_sugerida'] = self.dffinal2['nueva_compra_sugerida'].fillna(0)
        self.dffinal2['backorder'] = self.dffinal2['backorder'].fillna(0)
        
        mask = self.dffinal2['demanda_mensual'] != 0
        self.dffinal2.loc[mask, 'meses_proteccion'] = (
            self.dffinal2.loc[mask, 'meses_proteccion'] * 
            (self.dffinal2.loc[mask, 'compra_sugerida'] / self.dffinal2.loc[mask, 'demanda_mensual'])
        )
        
        columns_to_drop = ['codigo', 'nueva_compra_sugerida', 'caa']
        for col in columns_to_drop:
            if col in self.dffinal2.columns:
                self.dffinal2 = self.dffinal2.drop(columns=[col])

    def finalize_processing(self) -> None:
        self.dffinal2 = self.dffinal2.rename(columns={'caa_x': 'compras_recomendadas'})
        self.dffinal2 = self.dffinal2.drop_duplicates()
        self.dffinal2['compras_recomendadas'] = self.dffinal2['compras_recomendadas'].apply(lambda x: math.ceil(x / 50) * 50)
        self.dffinal2['costo_compra'] = self.dffinal2['monto_usd'] * self.dffinal2['compras_recomendadas']

        df1_final_filled = self.df1_final.fillna(0)
        df1_final_grouped = df1_final_filled.groupby(['articulo', 'fuente_suministro']).agg({
            'ingreso_usd_sin_recomendacion': 'sum',
            'ingreso_usd_con_recomendacion': 'sum'
        }).reset_index()

        self.dffinal2 = self.dffinal2.merge(
            df1_final_grouped[['articulo', 'fuente_suministro']], 
            how='left', 
            on='articulo'
        )

        df1_final_grouped['ganancia_oportunidad'] = (
            df1_final_grouped['ingreso_usd_con_recomendacion'] - 
            df1_final_grouped['ingreso_usd_sin_recomendacion']
        )

        df1_final_grouped_fs = df1_final_grouped.groupby(['fuente_suministro']).agg({
            'ganancia_oportunidad': 'sum'
        }).reset_index()

        df1_final_grouped_fs = df1_final_grouped_fs.sort_values(
            by='ganancia_oportunidad', 
            ascending=False
        ).reset_index(drop=True)
        
        df1_final_grouped_fs['hierarchy'] = df1_final_grouped_fs.index + 1

        self.dffinal2 = self.dffinal2.merge(
            df1_final_grouped_fs[['fuente_suministro', 'hierarchy']], 
            how='left', 
            on='fuente_suministro'
        )

        self.dffinal2 = self.dffinal2.sort_values(by='hierarchy')
        self.dffinal2 = self.dffinal2[[
            'articulo','stock','compras_recomendadas','demanda_mensual','meses_proteccion',
            'index_riesgo','riesgo','lt_x','mean_margen','ultima_fecha','monto_usd',
            'ultima_compra','costo_compra','fuente_suministro','hierarchy','backorder'
        ]]

    def process_all(self) -> None:
        self.load_data()
        self.preprocess_exchange_rates()
        self.get_currency_data()
        self.process_currency_data()
        self.merge_exchange_rates()
        self.process_inventory()
        self.merge_dataframes()
        self.calculate_risk()
        self.process_prices()
        self.calculate_margin()
        self.create_df1_final()
        self.create_final_dataframe()
        self.add_compra_real()
        self.finalize_processing()

if __name__ == "__main__":
    path = 'C:/Users/Christopher/OneDrive/Documentos/GitHub/catusita_revamp'
    processor = DataProcessor(path)
    processor.process_all()

[*********************100%%**********************]  4 of 4 completed


dffinal2 head:
      articulo   stock  compras_recomendadas  demanda_mensual  \
971  paw68a500  1449.0                  1850       922.833333   

     meses_proteccion  index_riesgo riesgo  lt_x  mean_margen ultima_fecha  \
971          0.410158      1.570164  Verde     2     0.533894   2024-09-21   

     monto_usd  ultima_compra  costo_compra         fuente_suministro  \
971      20.55          824.0       38017.5  prextoline - lubricantes   

     hierarchy  backorder  
971          4      500.0  


,articulo,stock,compras_recomendadas,demanda_mensual,meses_proteccion,index_riesgo,riesgo,lt_x,mean_margen,ultima_fecha,monto_usd,ultima_compra,costo_compra,fuente_suministro,hierarchy
384,dt-036,86.0,100,10.000000,7.716845,8.600000,Verde,6,0.366844,2023-05-05,22.700000,200.0,2270.000000,aisin - japon,1
676,ctx-071,731.0,5850,969.997864,0.153979,0.753610,Rojo,6,0.456828,2024-07-30,24.050000,180.0,140692.500000,aisin - japon,1
874,dn-047,34.0,50,5.000000,2.068043,6.800000,Verde,6,0.642040,2024-01-11,20.380000,50.0,1019.000000,aisin - japon,1
916,dtx-117,18.0,50,6.000000,10.734161,3.000000,Verde,6,0.684771,2024-04-16,28.650000,60.0,1432.500000,aisin - japon,1
155,dtx-099,68.0,150,19.421735,2.468459,3.501232,Verde,6,0.432788,2024-10-07,30.820000,100.0,4623.000000,aisin - japon,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,cf-500/1,19.0,100,22.000000,3.188737,0.863636,Rojo,4,0.560766,2024-09-12,10.480000,100.0,1048.000000,mann - brasil,33
1199,cf-300/1,0.0,100,24.500000,3.521156,0.000000,Rojo,4,0.544889,2024-09-12,7.650000,108.0,765.000000,mann - brasil,33
512,wk-1080/7x,4.0,200,45.250000,1.609876,0.088398,Rojo,4,0.344246,2024-09-24,16.678704,102.0,3335.740731,mann - brasil,33
1024,083865750,0.0,50,12.000000,1.387777,0.000000,Rojo,4,0.273668,NaT,NaN,NaN,NaN,monark - alemania,34


In [67]:
def preprocess_exchange_rates(tipo_de_cambio_df):
    tipo_de_cambio_df=tipo_de_cambio_df[['Código','Mnd','Fob','Ult. Fecha','Ult. Compra']]
    tipo_de_cambio_df.columns = ['codigo', 'moneda', 'monto', 'ultima_fecha', 'ultima_compra']
    tipo_de_cambio_df = tipo_de_cambio_df.copy()
    tipo_de_cambio_df['codigo'] = tipo_de_cambio_df['codigo'].astype(str)
    tipo_de_cambio_df = tipo_de_cambio_df.dropna(subset=['ultima_fecha'])
    tipo_de_cambio_df['codigo'] = tipo_de_cambio_df['codigo'].str.lower()
    tipo_de_cambio_df = tipo_de_cambio_df[tipo_de_cambio_df['ultima_fecha'].notna()]
    tipo_de_cambio_df['ultima_fecha'] = pd.to_datetime(tipo_de_cambio_df['ultima_fecha'], errors='coerce')
    return tipo_de_cambio_df

tipo_de_cambio_df = preprocess_exchange_rates(tipo_de_cambio_df)

In [68]:
def get_currency_data(tipo_de_cambio_df):
    start = tipo_de_cambio_df['ultima_fecha'].min().date()
    end = tipo_de_cambio_df['ultima_fecha'].max().date()
    currency_pairs = ['PENUSD=X', 'EURUSD=X', 'JPYUSD=X', 'GBPUSD=X']
    data = yf.download(currency_pairs, start=start, end=end)
    closing_prices = data['Close']
    closing_prices.columns = [col.split('.')[0] for col in closing_prices.columns]
    return closing_prices

closing_prices = get_currency_data(tipo_de_cambio_df)

[*********************100%%**********************]  4 of 4 completed


In [69]:
def process_currency_data(closing_prices):
    long_format = closing_prices.reset_index().melt(id_vars='Date', var_name='Currency Pair', value_name='Closing Price')
    long_format['Currency Pair'] = long_format['Currency Pair'].str.replace('=X', '', regex=False)
    long_format = long_format.dropna(subset=['Closing Price'])
    full_date_range = pd.date_range(start=long_format['Date'].min(), end=long_format['Date'].max(), freq='D')
    currency_pairs = long_format['Currency Pair'].unique()
    complete_index = pd.MultiIndex.from_product([full_date_range, currency_pairs], names=['Date', 'Currency Pair'])
    df_full = pd.DataFrame(index=complete_index).reset_index()
    long_format = df_full.merge(long_format, on=['Date', 'Currency Pair'], how='left')
    long_format['Closing Price'] = long_format.groupby('Currency Pair')['Closing Price'].fillna(method='ffill')
    long_format = long_format.rename(columns={'Closing Price': 'tc'})
    return long_format

long_format = process_currency_data(closing_prices)

In [70]:
def merge_exchange_rates(tipo_de_cambio_df, long_format):
    merged_df_tc = pd.merge(tipo_de_cambio_df, long_format, left_on='ultima_fecha', right_on='Date', how='left')
    merged_df_tc['monto'] = pd.to_numeric(merged_df_tc['monto'], errors='coerce')
    merged_df_tc['tc'] = pd.to_numeric(merged_df_tc['tc'], errors='coerce')
    
    def convert_to_usd(row):
        if pd.isna(row['Currency Pair']) or row['moneda'] == 'USD':
            return row['monto']
        currency_pair_map = {'SOL': 'PENUSD', 'EUR': 'EURUSD', 'JPY': 'JPYUSD', 'GBP': 'GBPUSD'}
        if row['moneda'] in currency_pair_map and row['Currency Pair'] == currency_pair_map[row['moneda']]:
            return row['monto'] / row['tc'] if row['moneda'] == 'SOL' else row['monto'] * row['tc']
        return 0

    merged_df_tc['monto_usd'] = merged_df_tc.apply(convert_to_usd, axis=1)
    merged_df_tc = merged_df_tc[merged_df_tc['monto_usd'] != 0]
    merged_df_tc_final = merged_df_tc[['codigo', 'ultima_fecha', 'monto_usd', 'ultima_compra']]
    merged_df_tc_final = merged_df_tc_final[merged_df_tc_final['monto_usd'].notna()]
    return merged_df_tc_final
merged_df_tc_final = merge_exchange_rates(tipo_de_cambio_df,long_format)

In [71]:
def process_inventory(df_inventory, results_models_comparison):
    df_inventory = df_inventory.copy()
    df_inventory.columns = ['cia', 'date', 'codigo', 'descripcion', 'um', 'stock']
    
    df_inventory = df_inventory[
        (df_inventory['date'] != 'Periodo') & 
        (df_inventory['date'].notna())
    ]

    df_inventory['date'] = pd.to_datetime(df_inventory['date'], format='%d/%m/%Y')
    max_date = results_models_comparison['date'].max()
    df_inventory = df_inventory[(df_inventory['date'] == max_date) & (df_inventory['codigo'].notna())]
    df_inventory.loc[:, 'codigo'] = df_inventory['codigo'].str.lower()
    return df_inventory
df_inventory = process_inventory(df_inventory, results_models_comparison)

In [72]:
results_models_comparison

,sku,lt,date,model,real,catusita,lookback_period,features_used,caa,caa_lt,corr_sd,loss
0,fc-1001,6,2024-12-01,xgboost,0,370.833333,6,"Menores_lag_5,Pesados_lag_5,Hibridos y Electri...",2375.994873,2406.601554,76.516701,30.675437
1,a-6020m,6,2024-12-01,linear,0,211.166667,6,"Pesados,Sistema de suspensión",986.131630,1001.131037,37.498518,32.067193
2,eo-17030,6,2024-12-01,median,0,355.666667,6,"Livianos,Menores_lag_6,Pesados_lag_6,Remolques...",2442.000000,2475.016090,82.540226,32.509670
3,54651-2h000,6,2024-12-01,es,0,23.000000,6,none,142.042902,148.348388,10.392305,32.712988
4,bdd020,6,2024-12-01,linear,0,157.000000,6,"Remolques y SemiR_lag_1,Pesados_lag_2,Menores_...",302.945844,334.707456,79.404030,33.319542
...,...,...,...,...,...,...,...,...,...,...,...,...
1279,dt-628,6,2024-12-01,mean,0,3.333333,6,"Livianos_lag_5,Lubricantes,Partes de Motor_lag...",169.200000,169.577124,0.942809,8228.626543
1280,48222,4,2024-12-01,mean,0,1211.666667,6,none,18320.000000,19232.945173,2282.362932,10559.596188
1281,48904,4,2024-12-01,es,0,108.333333,6,"Menores,Hibridos y Electricos_lag_2,Remolques ...",657.973900,683.733531,64.398930,24983.878017
1282,48951,4,2024-12-01,es,0,75.000000,6,"Hibridos y Electricos_lag_4,Menores_lag_4,Remo...",271.492209,287.891669,41.129876,25069.578879


In [73]:
df_inventory

,cia,date,codigo,descripcion,um,stock
0,TG,2024-12-01,b45a34900,AMORTIGUADOR DEL LH 338088,PZA,90
1,TG,2024-12-01,tto017,TERMINAL DEL EXT LH 45047-39156 CET-73,PZA,48
2,TG,2024-12-01,tto016,TERMINAL DEL EXT RH 45046-39256 CET-72,PZA,32
3,TG,2024-12-01,96424026sp,AMORTIGUADOR DEL LH 96424026SP,PZA,124
4,TG,2024-12-01,ky01-34-900a,AMORTIGUADOR DEL LH KY01-34-900A,PZA,32
...,...,...,...,...,...,...
4405,RJ,2024-12-01,tt-123,CRUCETA DE CARDAN GUT-23 04371-0K060,PZA,681
4406,RJ,2024-12-01,ty655183u,KIT DE EMBRAGUE TYC655 + TYD183U 275x180,JGO,25
4407,RJ,2024-12-01,tyc526,PLATO DE EMBRAGUE 260x170x298 TYD122U,PZA,7
4408,RJ,2024-12-01,tyc565,PLATO DE EMBRAGUE 236x150x271 TYD112U,PZA,99


In [74]:
df_products

,fecha,articulo,codigo,cantidad,transacciones,venta_pen,fuente_suministro,costo,lt
0,2021-01-04,48320,c08724,50.0,1,185.80,narva,134.870000,4.0
1,2021-01-04,48901,c08724,300.0,1,2165.20,narva,1413.050100,4.0
2,2021-01-04,c-23610,c10722,8.0,1,552.80,mann - brasil,310.150000,4.0
3,2021-01-05,zd1567,c12849,2.0,1,153.68,wagner - pastillas,96.560000,3.0
4,2021-01-06,rp2050100,c11327,90.0,1,3129.41,valvoline,2323.670040,2.0
...,...,...,...,...,...,...,...,...,...
669397,2024-08-24,dtx-099,c01687,6.0,1,1924.82,aisin - japon,1368.199998,6.0
669398,2024-10-02,ctx-071,c00890,6.0,1,1266.65,aisin - japon,882.769998,6.0
669399,2024-10-02,dt-100v,c00890,6.0,1,1266.65,aisin - japon,882.769998,6.0
669400,2024-10-02,ctx-062,c00890,2.0,1,434.82,aisin - japon,298.810000,6.0


In [75]:
def merge_dataframes(results_models_comparison, df_inventory, df_products):
    df_merged = results_models_comparison.copy()
    
    # Rename column
    df_merged = df_merged.rename(columns={'sku':'articulo'})
    
    # First merge with products
    df_merged = df_merged.merge(
        df_products[['articulo', 'fuente_suministro','lt']].drop_duplicates(), 
        how='left', 
        on='articulo'
    )
    
    # Convert date columns to datetime before merging
    df_merged['date'] = pd.to_datetime(df_merged['date'])
    df_inventory['date'] = pd.to_datetime(df_inventory['date'])
    
    # Merge with inventory
    df_merged = df_merged.merge(
        df_inventory[['codigo', 'stock', 'date']].drop_duplicates(), 
        how='left', 
        left_on=['articulo', 'date'], 
        right_on=['codigo', 'date']
    )
    
    # Fill NaN values and drop unnecessary column
    df_merged['stock'] = df_merged['stock'].fillna(0)
    df_merged = df_merged.drop(columns='codigo')
    
    return df_merged

df_merged = merge_dataframes(results_models_comparison, df_inventory, df_products)

In [76]:
df_merged

,articulo,lt_x,date,model,real,catusita,lookback_period,features_used,caa,caa_lt,corr_sd,loss,fuente_suministro,lt_y,stock
0,fc-1001,6,2024-12-01,xgboost,0,370.833333,6,"Menores_lag_5,Pesados_lag_5,Hibridos y Electri...",2375.994873,2406.601554,76.516701,30.675437,sakura filter,6.0,622.0
1,a-6020m,6,2024-12-01,linear,0,211.166667,6,"Pesados,Sistema de suspensión",986.131630,1001.131037,37.498518,32.067193,sakura filter,6.0,983.0
2,eo-17030,6,2024-12-01,median,0,355.666667,6,"Livianos,Menores_lag_6,Pesados_lag_6,Remolques...",2442.000000,2475.016090,82.540226,32.509670,sakura filter,6.0,2410.0
3,54651-2h000,6,2024-12-01,es,0,23.000000,6,none,142.042902,148.348388,10.392305,32.712988,ito koyama - amortiguadores,6.0,354.0
4,bdd020,6,2024-12-01,linear,0,157.000000,6,"Remolques y SemiR_lag_1,Pesados_lag_2,Menores_...",302.945844,334.707456,79.404030,33.319542,ishikawa - discos de freno,6.0,1016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,dt-628,6,2024-12-01,mean,0,3.333333,6,"Livianos_lag_5,Lubricantes,Partes de Motor_lag...",169.200000,169.577124,0.942809,8228.626543,aisin - japon,6.0,30.0
1283,48222,4,2024-12-01,mean,0,1211.666667,6,none,18320.000000,19232.945173,2282.362932,10559.596188,narva,4.0,14730.0
1284,48904,4,2024-12-01,es,0,108.333333,6,"Menores,Hibridos y Electricos_lag_2,Remolques ...",657.973900,683.733531,64.398930,24983.878017,narva,4.0,20.0
1285,48951,4,2024-12-01,es,0,75.000000,6,"Hibridos y Electricos_lag_4,Menores_lag_4,Remo...",271.492209,287.891669,41.129876,25069.578879,narva,4.0,40.0


In [77]:
def calculate_risk(df_merged):
    df_merged['index_riesgo'] = df_merged['stock'] / (df_merged['caa'] / df_merged['lt_x'])
    df_merged['riesgo'] = pd.cut(df_merged['index_riesgo'], bins=[-float('inf'), 1, 1.2, 1.5, float('inf')],
                                 labels=['Rojo', 'Naranja', 'Amarillo', 'Verde'], right=False)
    return df_merged
df_merged = calculate_risk(df_merged)

In [78]:
def process_prices(df_products):
    df_precio = df_products[['articulo', 'cantidad', 'venta_pen', 'fecha']].copy()
    df_precio['fecha'] = pd.to_datetime(df_precio['fecha'], errors='coerce')
    df_precio = df_precio[df_precio['fecha'].dt.year == 2024]
    df_precio['precio'] = df_precio['venta_pen'] / df_precio['cantidad']
    result = df_precio.groupby('articulo').agg(precio=('precio', 'mean')).reset_index()
    return result

result_precio = process_prices(df_products)

In [79]:
def calculate_margin(df_products):
    df_margen = df_products[['articulo', 'costo', 'venta_pen', 'fecha']].copy()
    df_margen['fecha'] = pd.to_datetime(df_margen['fecha'], errors='coerce')
    df_margen = df_margen[df_margen['fecha'].dt.year == 2024]
    df_margen['margen'] = df_margen['venta_pen'] / df_margen['costo'] - 1
    result = df_margen.groupby('articulo').agg(
        total_venta_pen=('venta_pen', 'sum'),
        mean_margen=('margen', 'mean')
    ).reset_index().sort_values(by='total_venta_pen', ascending=False)
    return result

margin_result = calculate_margin(df_products)

In [80]:
def create_df1_final(df_merged, result_precio, long_format):
    df1 = df_merged[['fuente_suministro', 'date', 'articulo','real', 'catusita', 'caa','lt_x']].copy()
    df1 = df1.rename(columns={'catusita': 'venta_sin_recomendacion', 'caa': 'venta_con_recomendacion'})
    df1_final = df1.merge(result_precio, how='left', on='articulo')
    df1_final = df1_final[['fuente_suministro', 'date', 'articulo', 'venta_sin_recomendacion', 'venta_con_recomendacion','real', 'precio','lt_x']]
    df1_final['ingreso_sin_recomendacion'] = np.where(df1_final['venta_sin_recomendacion']<df1_final['real'],df1_final['venta_sin_recomendacion'] * df1_final['precio'],df1_final['real'] * df1_final['precio'])
    df1_final['venta_con_recomendacion'] = np.where(df1_final['venta_con_recomendacion']<df1_final['real'],df1_final['venta_con_recomendacion'] * df1_final['precio'],df1_final['real'] * df1_final['precio'])
    df1_final['ingreso_sin_recomendacion_ajustado'] = df1_final['ingreso_sin_recomendacion'] / (df1_final['lt_x'] * 0.83)
    df1_final['ingreso_con_recomendación_ajustado'] = df1_final['venta_con_recomendacion'] / (df1_final['lt_x'] * 0.83)
    
    penusd_tc = long_format[long_format['Currency Pair'] == 'PENUSD'].groupby('Date')['tc'].last().reset_index()
    df1_final = df1_final.merge(penusd_tc, how='left', left_on='date', right_on='Date')
    df1_final['tc']=1/df1_final['tc']
    df1_final['ingreso_usd_sin_recomendacion'] = df1_final['ingreso_sin_recomendacion_ajustado'] / df1_final['tc']
    df1_final['ingreso_usd_con_recomendacion'] = df1_final['ingreso_con_recomendación_ajustado'] / df1_final['tc']
    df1_final = df1_final[['fuente_suministro', 'date', 'articulo', 'lt_x', 'ingreso_usd_sin_recomendacion', 'ingreso_usd_con_recomendacion', 'tc']]
    return df1_final.drop_duplicates()

df1_final = create_df1_final(df_merged, result_precio, long_format)

In [81]:
def create_final_dataframe(df_merged, margin_result, merged_df_tc_final):
    last_date = df_merged['date'].max()
    df_merged_last = df_merged[df_merged['date'] == last_date].copy()
    
    df_merged_last['demanda_mensual'] = df_merged_last['caa'] / df_merged_last['lt_x']
    dffinal2 = df_merged_last[['articulo', 'stock', 'caa', 'demanda_mensual', 'corr_sd', 'index_riesgo', 'riesgo', 'lt_x']]
    dffinal2 = dffinal2.copy()
    dffinal2['meses_proteccion'] = dffinal2['corr_sd'] / dffinal2['demanda_mensual']
    dffinal2 = dffinal2[['articulo', 'stock', 'caa', 'demanda_mensual', 'meses_proteccion', 'index_riesgo', 'riesgo', 'lt_x']]
    dffinal2 = dffinal2.merge(margin_result[['articulo', 'mean_margen']], how='left', on='articulo')
    dffinal2 = dffinal2.merge(merged_df_tc_final, how='left', left_on='articulo', right_on='codigo')
    return dffinal2
dffinal2 = create_final_dataframe(df_merged, margin_result, merged_df_tc_final)

In [82]:
dffinal2

,articulo,stock,caa,demanda_mensual,meses_proteccion,index_riesgo,riesgo,lt_x,mean_margen,codigo,ultima_fecha,monto_usd,ultima_compra
0,fc-1001,622.0,2375.994873,395.999146,0.193224,1.570710,Verde,6,0.686879,fc-1001,2024-09-30,2.37,500.0
1,fc-1001,622.0,2375.994873,395.999146,0.193224,1.570710,Verde,6,0.686879,fc-1001,2024-09-30,2.37,500.0
2,fc-1001,622.0,2375.994873,395.999146,0.193224,1.570710,Verde,6,0.686879,fc-1001,2024-09-30,2.37,500.0
3,fc-1001,622.0,2375.994873,395.999146,0.193224,1.570710,Verde,6,0.686879,fc-1001,2024-09-30,2.37,500.0
4,a-6020m,983.0,986.131630,164.355272,0.228155,5.980946,Verde,6,1.648466,a-6020m,2024-10-16,8.10,208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,rxfc113644bep,0.0,459.797571,229.898786,2.726110,0.000000,Rojo,2,0.267835,rxfc113644bep,2024-10-09,4.86,1728.0
5004,rxfc113644bep,0.0,459.797571,229.898786,2.726110,0.000000,Rojo,2,0.267835,rxfc113644bep,2024-10-09,4.86,1728.0
5005,rxfc113644bep,0.0,459.797571,229.898786,2.726110,0.000000,Rojo,2,0.267835,rxfc113644bep,2024-10-09,4.86,1728.0
5006,rxfc113644bep,0.0,459.797571,229.898786,2.726110,0.000000,Rojo,2,0.267835,rxfc113644bep,2024-10-09,4.86,1728.0


In [83]:
df_merged

,articulo,lt_x,date,model,real,catusita,lookback_period,features_used,caa,caa_lt,corr_sd,loss,fuente_suministro,lt_y,stock,index_riesgo,riesgo
0,fc-1001,6,2024-12-01,xgboost,0,370.833333,6,"Menores_lag_5,Pesados_lag_5,Hibridos y Electri...",2375.994873,2406.601554,76.516701,30.675437,sakura filter,6.0,622.0,1.570710,Verde
1,a-6020m,6,2024-12-01,linear,0,211.166667,6,"Pesados,Sistema de suspensión",986.131630,1001.131037,37.498518,32.067193,sakura filter,6.0,983.0,5.980946,Verde
2,eo-17030,6,2024-12-01,median,0,355.666667,6,"Livianos,Menores_lag_6,Pesados_lag_6,Remolques...",2442.000000,2475.016090,82.540226,32.509670,sakura filter,6.0,2410.0,5.921376,Verde
3,54651-2h000,6,2024-12-01,es,0,23.000000,6,none,142.042902,148.348388,10.392305,32.712988,ito koyama - amortiguadores,6.0,354.0,14.953229,Verde
4,bdd020,6,2024-12-01,linear,0,157.000000,6,"Remolques y SemiR_lag_1,Pesados_lag_2,Menores_...",302.945844,334.707456,79.404030,33.319542,ishikawa - discos de freno,6.0,1016.0,20.122408,Verde
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,dt-628,6,2024-12-01,mean,0,3.333333,6,"Livianos_lag_5,Lubricantes,Partes de Motor_lag...",169.200000,169.577124,0.942809,8228.626543,aisin - japon,6.0,30.0,1.063830,Naranja
1283,48222,4,2024-12-01,mean,0,1211.666667,6,none,18320.000000,19232.945173,2282.362932,10559.596188,narva,4.0,14730.0,3.216157,Verde
1284,48904,4,2024-12-01,es,0,108.333333,6,"Menores,Hibridos y Electricos_lag_2,Remolques ...",657.973900,683.733531,64.398930,24983.878017,narva,4.0,20.0,0.121585,Rojo
1285,48951,4,2024-12-01,es,0,75.000000,6,"Hibridos y Electricos_lag_4,Menores_lag_4,Remo...",271.492209,287.891669,41.129876,25069.578879,narva,4.0,40.0,0.589336,Rojo


In [84]:
def add_compra_real(dffinal2, back_order, path):
    # Load data with debugging prints
    print("Loading data...")
    df_predicciones = pd.read_csv(f"{path}/data/cleaned/predictions.csv")
    df_inventory2 = pd.read_excel(f"{path}/data/raw/catusita/inventory.xlsx")
    
    print("Inventory columns:", df_inventory2.columns.tolist())
    
    # Rename sku to articulo in df_predicciones
    df_predicciones = df_predicciones.rename(columns={'sku': 'articulo'})
    
    # Process inventory data
    df_inventory2 = df_inventory2[
        (df_inventory2['FECHA AL'] != 'Periodo') & 
        (df_inventory2['FECHA AL'].notna())
    ]
    
    # Convert dates
    df_inventory2['FECHA AL'] = pd.to_datetime(df_inventory2['FECHA AL'], format='%d/%m/%Y')
    max_date = df_inventory2['FECHA AL'].max()
    df_inventory2 = df_inventory2[df_inventory2['FECHA AL'] == max_date]
    df_inventory2['FECHA AL'] = df_inventory2['FECHA AL'].dt.strftime('%d/%m/%Y')
    df_inventory2['CODIGO'] = df_inventory2['CODIGO'].str.lower()
    
    # Process predictions dates
    df_predicciones['date'] = pd.to_datetime(df_predicciones['date'], format='%Y-%m-%d')
    df_predicciones['date'] = df_predicciones['date'].dt.strftime('%d/%m/%Y')
    
    # First merge with inventory data
    print("Merging with inventory data...")
    merged_df = df_predicciones.merge(
        df_inventory2[['FECHA AL', 'CODIGO', 'STOCK']], 
        how='left', 
        left_on=['articulo'], 
        right_on=['CODIGO']
    )
    
    merged_df['STOCK'] = merged_df['STOCK'].fillna(0)
    
    # Merge with back_order
    print("Merging with back_order data...")
    merged_df = merged_df.merge(
        back_order, 
        how='left', 
        on='articulo'
    )
    
    # Process the merged data
    merged_df['backorder'] = merged_df['backorder'].fillna(0)
    
    # Calculate intermediary values
    merged_df['sobrante'] = np.maximum(merged_df['STOCK'] + merged_df['backorder'] - merged_df['caa_lt'], 0)
    merged_df['nueva_compra_sugerida'] = np.maximum(merged_df['caa'] - merged_df['sobrante'], 0)
    merged_df['nueva_compra_sugerida'] = np.ceil(merged_df['nueva_compra_sugerida']).astype(int)
    
    # Prepare final merge columns
    merge_columns = merged_df[['articulo', 'nueva_compra_sugerida', 'caa']].copy()
    
    # Final merges and adjustments
    print("Performing final merges...")
    result_df = dffinal2.merge(
        merge_columns,
        how='left',
        on='articulo'
    )
    
    # Update compra_sugerida with the new values
    result_df['compra_sugerida'] = result_df['nueva_compra_sugerida'].fillna(0)
    
    # Calculate meses_proteccion using the original demanda_estimada
    mask = result_df['demanda_estimada'] != 0
    result_df.loc[mask, 'meses_proteccion'] = (
        result_df.loc[mask, 'meses_proteccion'] * 
        (result_df.loc[mask, 'compra_sugerida'] / result_df.loc[mask, 'demanda_estimada'])
    )
    
    # Clean up columns
    columns_to_drop = [
        'codigo', 'nueva_compra_sugerida', 'caa'
    ]
    for col in columns_to_drop:
        if col in result_df.columns:
            result_df = result_df.drop(columns=[col])
    
    print("Processing complete.")
    return result_df
try:
    dffinal2 = add_compra_real(dffinal2, back_order, path)
except Exception as e:
    print(f"Error occurred: {str(e)}")

Loading data...
Inventory columns: ['CIA', 'FECHA AL', 'CODIGO', 'DESCRIPCION', 'UM', 'STOCK']
Merging with inventory data...
Merging with back_order data...
Performing final merges...
Error occurred: 'demanda_estimada'


In [86]:
# Initial operations on dffinal2
dffinal2 = dffinal2.rename(columns={'caa': 'compras_recomendadas'})
dffinal2 = dffinal2.drop_duplicates()
dffinal2['compras_recomendadas'] = dffinal2['compras_recomendadas'].apply(lambda x: math.ceil(x / 50) * 50)
dffinal2['costo_compra'] = dffinal2['monto_usd'] * dffinal2['compras_recomendadas']

# Fill NA values in df1_final
df1_final_filled = df1_final.fillna(0)

# Group df1_final by articulo and fuente_suministro
df1_final_grouped = df1_final_filled.groupby(['articulo', 'fuente_suministro']).agg({
    'ingreso_usd_sin_recomendacion': 'sum',
    'ingreso_usd_con_recomendacion': 'sum'
}).reset_index()

# Merge dffinal2 with df1_final_grouped
dffinal2 = dffinal2.merge(
    df1_final_grouped[['articulo', 'fuente_suministro']], 
    how='left', 
    on='articulo'
)

# Calculate ganancia_oportunidad
df1_final_grouped['ganancia_oportunidad'] = (
    df1_final_grouped['ingreso_usd_con_recomendacion'] - 
    df1_final_grouped['ingreso_usd_sin_recomendacion']
)

# Group by fuente_suministro
df1_final_grouped_fs = df1_final_grouped.groupby(['fuente_suministro']).agg({
    'ganancia_oportunidad': 'sum'
}).reset_index()

# Sort and add hierarchy
df1_final_grouped_fs = df1_final_grouped_fs.sort_values(
    by='ganancia_oportunidad', 
    ascending=False
).reset_index(drop=True)
df1_final_grouped_fs['hierarchy'] = df1_final_grouped_fs.index + 1

# Final merge with hierarchy information
dffinal2 = dffinal2.merge(
    df1_final_grouped_fs[['fuente_suministro', 'hierarchy']], 
    how='left', 
    on='fuente_suministro'
)

# Final sort
dffinal2 = dffinal2.sort_values(by='hierarchy')

# Print results
print("df1_final head:")
print(df1_final.head())
print("\ndffinal2 head:")
print(dffinal2.head())

df1_final head:
             fuente_suministro       date     articulo  lt_x  \
0                sakura filter 2024-12-01      fc-1001     6   
1                sakura filter 2024-12-01      a-6020m     6   
2                sakura filter 2024-12-01     eo-17030     6   
3  ito koyama - amortiguadores 2024-12-01  54651-2h000     6   
4   ishikawa - discos de freno 2024-12-01       bdd020     6   

   ingreso_usd_sin_recomendacion  ingreso_usd_con_recomendacion  tc  
0                            NaN                            NaN NaN  
1                            NaN                            NaN NaN  
2                            NaN                            NaN NaN  
3                            NaN                            NaN NaN  
4                            NaN                            NaN NaN  

dffinal2 head:
    articulo  stock  compras_recomendadas  demanda_mensual  meses_proteccion  \
384   dt-036   86.0                   100        10.000000          1.837344   
676

In [87]:
dffinal2

,articulo,stock,compras_recomendadas,demanda_mensual,meses_proteccion,index_riesgo,riesgo,lt_x,mean_margen,codigo,ultima_fecha,monto_usd,ultima_compra,costo_compra,fuente_suministro,hierarchy
384,dt-036,86.0,100,10.000000,1.837344,8.600000,Verde,6,0.366844,dt-036,2023-05-05,22.700000,200.0,2270.000000,aisin - japon,1
676,ctx-071,731.0,5850,969.997864,0.025663,0.753610,Rojo,6,0.456828,ctx-071,2024-07-30,24.050000,180.0,140692.500000,aisin - japon,1
874,dn-047,34.0,50,5.000000,0.382971,6.800000,Verde,6,0.642040,dn-047,2024-01-11,20.380000,50.0,1019.000000,aisin - japon,1
916,dtx-117,18.0,50,6.000000,1.789027,3.000000,Verde,6,0.684771,dtx-117,2024-04-16,28.650000,60.0,1432.500000,aisin - japon,1
155,dtx-099,68.0,150,19.421735,0.409759,3.501232,Verde,6,0.432788,dtx-099,2024-10-07,30.820000,100.0,4623.000000,aisin - japon,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,cf-500/1,19.0,100,22.000000,0.797184,0.863636,Rojo,4,0.560766,cf-500/1,2024-09-12,10.480000,100.0,1048.000000,mann - brasil,33
1199,cf-300/1,0.0,100,24.500000,0.880289,0.000000,Rojo,4,0.544889,cf-300/1,2024-09-12,7.650000,108.0,765.000000,mann - brasil,33
512,wk-1080/7x,4.0,200,45.250000,0.402469,0.088398,Rojo,4,0.344246,wk-1080/7x,2024-09-24,16.678704,102.0,3335.740731,mann - brasil,33
1024,083865750,0.0,50,12.000000,0.346944,0.000000,Rojo,4,0.273668,NaN,NaT,NaN,NaN,NaN,monark - alemania,34
